In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import csv
import matplotlib.pyplot as plt
from scipy import signal
import seaborn as sns
import sklearn
import impyute

import os
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import OneHotEncoder
from numpy.random import seed

from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
import xgboost

In [2]:
df=pd.read_csv('PreProcessed_Train_Data.csv')
Tdf=pd.read_csv('PreProcessed_Test_Data.csv')

In [3]:
Features=['A2', 'A5', 'A7', 'A8', 'A10', 'A11', 'A12', 'A13', 'A14', 'A1_a',
       'A3_l', 'A3_y', 'A4_gg', 'A4_p', 'A6_aa', 'A6_c', 'A6_cc', 'A6_e',
       'A6_ff', 'A6_i', 'A6_j', 'A6_k', 'A6_m', 'A6_q', 'A6_r', 'A6_w', 'A6_x',
       'A9_bb', 'A9_dd', 'A9_ff', 'A9_h', 'A9_j', 'A9_n', 'A9_o', 'A9_z',
       'A15_g', 'A15_p', 'A15_s']

X=df.loc[:,Features]
Y=df.loc[:,['A16']]

In [4]:
## Hyper Parameter Optimization

params={
 "learning_rate"    : [ 0.05, 0.10, 0.15, 0.20, 0.25, 0.30 ] ,
 "max_depth"        : [ 3, 4, 5, 6, 8,9, 10, 12,13,14, 15],
 "min_child_weight" : [ 1, 3, 5, 7,8,9,10],
 "gamma"            : [ 0.0, 0.1, 0.2 , 0.3, 0.4,0.5,0.6 ],
 "colsample_bytree" : [ 0.3, 0.35,0.4,0.45,0.5 , 0.7 ],
 "random_state"     : [i for i in range (0,25,1)]
    
}

In [5]:
classifier=xgboost.XGBClassifier()
random_search=RandomizedSearchCV(classifier,param_distributions=params,n_iter=125,scoring='roc_auc',n_jobs=-1,cv=10)
random_search.fit(X,Y)

C:\Users\Imesh Ekanayake\.conda\envs\Tensorflow\lib\site-packages\sklearn\preprocessing\_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Imesh Ekanayake\.conda\envs\Tensorflow\lib\site-packages\sklearn\preprocessing\_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


RandomizedSearchCV(cv=10, error_score=nan,
                   estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                           colsample_bylevel=1,
                                           colsample_bynode=1,
                                           colsample_bytree=1, gamma=0,
                                           learning_rate=0.1, max_delta_step=0,
                                           max_depth=3, min_child_weight=1,
                                           missing=None, n_estimators=100,
                                           n_jobs=1, nthread=None,
                                           objective='binary:logistic',
                                           random_state=0, reg_alpha=0,
                                           reg_lambda=1, s...
                                                             0.45, 0.5, 0.7],
                                        'gamma': [0.0, 0.1, 0.2, 0.3, 0.4, 0.5,
                   

In [6]:
random_search.best_estimator_

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.3, gamma=0.5,
              learning_rate=0.15, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=19,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [9]:
classifier=xgboost.XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.3, gamma=0.5,
              learning_rate=0.15, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=19,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

from sklearn.model_selection import cross_val_score
score=cross_val_score(classifier,X,Y,cv=10)

print(score.mean())
score

C:\Users\Imesh Ekanayake\.conda\envs\Tensorflow\lib\site-packages\sklearn\preprocessing\_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Imesh Ekanayake\.conda\envs\Tensorflow\lib\site-packages\sklearn\preprocessing\_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.8574999999999999


array([0.71428571, 0.66071429, 0.98181818, 0.96363636, 0.90909091,
       0.89090909, 0.92727273, 0.85454545, 0.94545455, 0.72727273])

In [10]:
Model_F=classifier


TX=Tdf.loc[:,Features]
Model_F.fit(X,Y)

res=list(Model_F.predict(Tdf))
p=[]
new_att=["Id","Category"]
count=0
for i in res:
    count+=1
    if i==1:p.append([count,"Success"])
    else:p.append([count,"Failure"])

p=np.array(p)
Submit= pd.DataFrame(data=np.array(p),columns=new_att)


Submit.to_csv(r'Z:\ML\Project - 6th sem\Submit_10_4_4.csv', index = False)